<font color=orange><h5><b>[REMINDER: MAKE A COPY OF THIS NOTEBOOK, DO NOT EDIT]</b></h1></font>

In [43]:
#@title Instructor Tokens
#@markdown Use these sparingly, as only one person can use each at a time!

!ngrok authtoken 2esTEyBVYVUkNTh0ETgpZq6ThNH_RzyMevymuyYSKd76mSY7
# !ngrok authtoken 2esTQObOz9XZ2be2DF4aPlS3upu_88X3yovgR3x3NFNAVTvVf
# !ngrok authtoken 2PfMYFZgmz9KLqLlZ4Pn7Bqa8ZA_3vMo1vufxRTWJLHhDHUBC

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# **ML Deployment: Planet Hunters**

<img src="https://www.eso.org/public/archives/images/screen/eso2202a.jpg" width="730" height="450" />

In [2]:
#@title Run this to setup libraries and environment
import os
import sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

with HiddenPrints():

    # Installing Streamlit & pyngrok
    !pip -q install streamlit
    !pip -q install pyngrok
    from pyngrok import ngrok
    import streamlit as st
    !wget "https://raw.githubusercontent.com/NolanChai/model_repo/main/income.csv"

def launch_website():
  print ("Click this link to try your web app:")
  if (ngrok.get_tunnels() != None):
    ngrok.kill()
  public_url = ngrok.connect()
  print (public_url)
  !streamlit run --server.port 80 app.py >/dev/null

import matplotlib.pyplot as plt
%matplotlib inline

from urllib.request import urlretrieve
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

!wget -q --show-progress 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Planet%20Hunters/exoTrain.csv'
!wget -q --show-progress 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Planet%20Hunters/exoTest.csv'

import seaborn as sns
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn import  metrics
from sklearn import tree
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from scipy.signal import savgol_filter
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,ConfusionMatrixDisplay,precision_score,recall_score,f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, normalize

import tensorflow as tf
import keras
from keras.models import Sequential
from keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, Conv2D, MaxPooling2D, BatchNormalization, MaxPooling1D
from keras.losses import categorical_crossentropy
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import plotly.graph_objects as go

df_train = pd.read_csv('exoTrain.csv')
df_train['LABEL'] = df_train['LABEL'] -1
df_test = pd.read_csv('exoTest.csv')
df_test['LABEL'] = df_test['LABEL'] - 1

def plot_graphs(history, best):

  plt.figure(figsize=[10,4])
  # summarize history for accuracy
  plt.subplot(121)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy across training\n best accuracy of %.02f'%best[1])
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')

  # summarize history for loss
  plt.subplot(122)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss across training\n best loss of %.02f'%best[0])
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

def analyze_results(model, X_train, y_train, X_test, y_test):
    """
    Helper function to help interpret and model performance.

    Args:
    model: estimator instance
    X_train: {array-like, sparse matrix} of shape (n_samples, n_features)
    Input values for model training.
    y_train : array-like of shape (n_samples,)
    Target values for model training.
    X_test: {array-like, sparse matrix} of shape (n_samples, n_features)
    Input values for model testing.
    y_test : array-like of shape (n_samples,)
    Target values for model testing.

    Returns:
    None
    """
    print("-------------------------------------------")
    print("Model Results")
    print("")
    print("Training:")
    if type(model) == keras.src.engine.sequential.Sequential:
      train_predictions = model.predict(X_train)
      train_predictions = (train_predictions > 0.5)
      cm = confusion_matrix(y_train, train_predictions)
      labels = [0, 1]
      df_cm = pd.DataFrame(cm,index = labels,columns = labels)
      fig = plt.figure()
      res = sns.heatmap(df_cm, annot=True,cmap='Blues', fmt='g')
      #plt.yticks([1.25, 3.75], labels,va='center')
      plt.title('Confusion Matrix - Test Data')
      plt.ylabel('True label')
      plt.xlabel('Predicted label')
      plt.show()
    else:
      plt.close()
      ConfusionMatrixDisplay.from_estimator(model,X_train,y_train)
      plt.show()

    print("Testing:")
    if type(model) == keras.src.engine.sequential.Sequential:
      test_predictions = model.predict(X_test)
      test_predictions = (test_predictions > 0.5)
      cm = confusion_matrix(y_test, test_predictions)
      labels = [0, 1]
      df_cm = pd.DataFrame(cm,index = labels,columns = labels)
      fig = plt.figure()
      res = sns.heatmap(df_cm, annot=True,cmap='Blues', fmt='g')
      #plt.yticks([1.25, 3.75], labels,va='center')
      plt.title('Confusion Matrix - Test Data')
      plt.ylabel('True label')
      plt.xlabel('Predicted label')
      plt.show()
    else:
      ConfusionMatrixDisplay.from_estimator(model,X_test,y_test)

X_train = df_train.drop('LABEL', axis=1)
y_train = df_train['LABEL']
X_test = df_test.drop('LABEL', axis=1)
y_test = df_test['LABEL']

# Preprocess Data
# Helper functions that we can run for the three augmentation functions that will be used, but not explroed in depth

def smote(a,b):
    model = SMOTE()
    X,y = model.fit_resample(a, b)
    return X,y

def savgol(df1,df2):
    x = savgol_filter(df1,21,4,deriv=0)
    y = savgol_filter(df2,21,4,deriv=0)
    return x,y

def fourier(df1,df2):
    X_train = np.abs(np.fft.fft(df1, axis=1))
    X_test = np.abs(np.fft.fft(df2, axis=1))
    return X_train,X_test

def norm(df1,df2):
    X_train = normalize(df1)
    X_test = normalize(df2)
    return X_train,X_test

def robust(df1,df2):
    scaler = RobustScaler()
    X_train = scaler.fit_transform(df1)
    X_test = scaler.transform(df2)
    return X_train,X_test

fourier_X_train, fourier_X_test = fourier(X_train, X_test)
savgol_X_train, savgol_X_test = savgol(fourier_X_train, fourier_X_test)
norm_X_train, norm_X_test = norm(savgol_X_train,savgol_X_test)
robust_X_train, robust_X_test = robust(norm_X_train, norm_X_test)
smote_X_train,smote_y_train = smote(robust_X_train, y_train)

# Here we're adding the generated, augmented data onto the testing data
# aug_X_train, new_X_test_data, aug_y_train, new_y_test_data = train_test_split(smote_X_train, smote_y_train, test_size=0.3)
# aug_X_test = np.concatenate((robust_X_test, new_X_test_data), axis=0)
# aug_y_test = np.concatenate((y_test, new_y_test_data), axis=0)

def preprocess_data(df):
    """Apply preprocessing steps to the dataframe."""
    X = df.drop('LABEL', axis=1).values
    y = df['LABEL'].values
    # Fourier transform
    X = np.abs(np.fft.fft(X, axis=1))
    # Savitzky-Golay filter
    X = savgol_filter(X, 21, 4, deriv=0, axis=1)
    # Normalize
    X = normalize(X)
    # Robust scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    # SMOTE
    smote = SMOTE()
    X, y = smote.fit_resample(X, y)
    # Expand dimensions for CNN input
    X_cnn = np.expand_dims(X, axis=2)
    return X, X_cnn, y

exoTrain.csv        100%[===================>] 250.08M   113MB/s    in 2.2s    
exoTest.csv         100%[===================>]  27.57M   122MB/s    in 0.2s    


Great work! We've been able to develop and compare various models to identify potential habitable planets from photometric data - now we can try deploying them to the web! While your site is running, you'll be able to use it on your laptop or computer or share it with friends! We'll be using [Streamlit](https://www.streamlit.io/), a library of website objects and methods that allows us to quickly build a site.

# **Part 1. Streamlit - Deploying your model to the web**

The goal of this session is to learn how to deploy the models that we have been training to the web so they can be shared with the world!

Let's start with an example.

Check out [this example](https://bgremoval.streamlit.app/) and answer the following
**questions:**
* Who is this application for?
* How does the user input data - are these intuitive ways of interacting with the app?
* What does the application do with the data?
* Evaluate the ease of use and look of the application.

(More cool examples [here](https://streamlit.io/gallery?category=favorites)!)

Streamlit interprets Python files as a website! This is great for several reasons
* No need to know HTML, CSS, Javascript,... etc
* Easy to use our trained models which are already in Python!

We'll write everything to a file called app.py, which is used for Streamlit to launch the site.

```
%%writefile app.py
```
The `%%writefile` command writes to a file (and creates one if it doesn't already exist!). Everything that follows the rest of this block of code will be written to `app.py`. <br> In this case, we use this command to create our `app.py` file, which you can check in the 'Files' tab in left sidebar.

In [34]:
%%writefile app.py
import streamlit as st
st.title('Hello World!')

Overwriting app.py


To launch our `app.py`, we need to use a combination of Streamlit and ngrok. Ngrok is a server hosting service, and Streamlit is a library that allows us to connect our Python code to Ngrok. <br> To get access to our Ngrok server, we need to sign up on their website and get a unique **authentication token**.

To launch our `app.py`, we need to use a combination of Streamlit and ngrok. Ngrok serves as a hosting service that enables us to establish secure tunnels to our localhost, making our local server accessible over the internet. Streamlit, on the other hand, is a powerful library designed to turn Python scripts into interactive web applications easily. Together, Ngrok allows us to host our Streamlit code publicly on the web!

<br> To get access to our Ngrok server, we need to sign up on their website and get a unique **authentication token**.

<font color=SlateGrey><h2><b>
Use [these](https://drive.google.com/file/d/12zwuOuKh91VSHIHS-6S4ADF4HLC2wKJq/view?usp=sharing) instructions to create a ngrok account and get your authtoken!
</b></h2></font>

<font color=DarkGray><h3><b>
Paste your authtoken below next to `!ngrok authtoken`!
</b></h3></font>

In [45]:
!ngrok authtoken 2jqZZ5QOAyg7ndiavkl74putT7V_tiXgYZ24dz1UkRB9S7uD

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Now, we can launch our website through the `launch_website()` function, which connects our ngrok token by building a 'tunnel' to our Streamlit code.

We can use the following launch_website() function to deploy our `app.py` file. Click on the first link, and hit the `Visit Site` button to view your site!

In [46]:
launch_website()

Click this link to try your web app:
NgrokTunnel: "https://325b-35-185-1-182.ngrok-free.app" -> "http://localhost:80"


<font color=Blue><b>
Congratulations, you have your first working site!
</b></font>

Now the question remains, how do we connect our models and data to our website?

## **Step 2: Loading the model**

Let's load our previously trained model! :)

Here is a reference on how to save and load sklearn and tensorflow models!

For sklearn:
```
from joblib import dump, load

# ====== Save model ========
dump(model, 'filename.joblib')

# ====== Load model ========
clf = load('filename.joblib')
```

For tensorflow:
```
import tensorflow as tf

# ====== Save model ========
model.save("filename.h5")

# ====== Load model ========
tf.keras.models.load_model("filename.h5")
```

The CNN model we worked on last time is a tensorflow model! Let's load in the model we created last time:




In [47]:
#@title Run this cell to load the model

from google.colab import drive
drive.mount('/content/gdrive')
cnn_path = '/content/gdrive/My Drive/cnn.zip'

import zipfile

with zipfile.ZipFile(cnn_path, 'r') as zip_ref:
    zip_ref.extractall('')
model = tf.keras.models.load_model('cnn')
model.summary()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3197, 8)           48        
                                                                 
 max_pooling1d (MaxPooling1  (None, 800, 8)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 800, 16)           400       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 200, 16)           0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 3200)              

And preprocess our data before loading it into our site:

In [49]:
X_train, X_train_cnn, y_train = preprocess_data(df_train)

Recall a few of the visualizations we've done so far:
* Light Curve Graphs
* Confusion Matrices

## **Step 3: Graphs**
Now that we have our model imported, we need to consider a few ways the user might be able to interact with it and the results that we've found thus far! One of the best (and easiest) ways to do this is through graphs. Streamlit works best with plotly graphs, a similar yet more visually appealing library of graph functions to matplotlib.

### Revisiting Light Curve Graphs

We've gone ahead and imported the light curve visualizations from our past notebooks into a Streamlit site.

**After changing the filename to your saved model's path,** go ahead and give it a try!

In [50]:
%%writefile app.py
################################################################################
##########=           (Setup Code from Previous NBs)           =################
################################################################################
import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from scipy.signal import savgol_filter
from sklearn.preprocessing import normalize

# Load data
df_train = pd.read_csv('exoTrain.csv')
df_train['LABEL'] = df_train['LABEL'] - 1

# Load the CNN model
model = tf.keras.models.load_model('cnn')

def preprocess_data(df):
    """Apply preprocessing steps to the dataframe."""
    X = df.drop('LABEL', axis=1).values
    y = df['LABEL'].values
    # Fourier transform
    X = np.abs(np.fft.fft(X, axis=1))
    # Savitzky-Golay filter
    X = savgol_filter(X, 21, 4, deriv=0, axis=1)
    # Normalize
    X = normalize(X)
    # Robust scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    # SMOTE
    smote = SMOTE()
    X, y = smote.fit_resample(X, y)
    # Expand dimensions for CNN input
    X = np.expand_dims(X, axis=2)
    return X, y

X_train, y_train = preprocess_data(df_train)

def predict(index):
    """Make a prediction using the Keras model."""
    tensor = X_train[index].reshape(1, -1, 1)
    output = model.predict(tensor)
    return output.flatten()[0]

################################################################################
##########=                   (Light Curves)                   =################
################################################################################
st.title('Exoplanet Light Curve Visualization with CNN Predictions')

# Slider for selecting the index of the light curve
index = st.slider("Select Index for Light Curve", min_value=0, max_value=len(X_train)-1, value=12, step=1)

# Display CNN prediction results
prediction = predict(index)
st.write(f"Prediction (probability of being an exoplanet): {prediction:.4f}")
t_0 = st.slider("Start of Period (t_0)", min_value=0, max_value=3197, value=430, step=1)
period = st.slider("Length of Period", min_value=0, max_value=3197, value=1184, step=1)

# Create Plotly graph for the full light curve with a rectangle highlighting the period
fig = go.Figure()
fig.add_trace(go.Scatter(y=X_train[index].flatten(), mode='lines', name='Light Curve'))
fig.add_shape(type="rect",
              x0=t_0, y0=min(X_train[index])-5,
              x1=t_0+period, y1=max(X_train[index])+5,
              line=dict(color="Red"),
              fillcolor="LightPink", opacity=0.5)
fig.update_layout(title="Box Covering One Period of Exoplanet Transit",
                  xaxis_title="Observation Point",
                  yaxis_title="Normalized Flux",
                  showlegend=True)
st.plotly_chart(fig, use_container_width=True)

# Create Plotly graph for just the selected period
fig_period = go.Figure()
fig_period.add_trace(go.Scatter(y=X_train[index, t_0: t_0+period].flatten(), mode='lines', name='Selected Period'))
fig_period.update_layout(title="Plot of Just One Period",
                         xaxis_title="Observation Point",
                         yaxis_title="Normalized Flux",
                         showlegend=True)
st.plotly_chart(fig_period, use_container_width=True)


Overwriting app.py


**Launch your website again:**

In [10]:
launch_website()

Click this link to try your web app:
NgrokTunnel: "https://060c-35-185-1-182.ngrok-free.app" -> "http://localhost:80"


We can do the same thing for our confusion matrices as well!

In [51]:
%%writefile app.py
model_path = 'Planet_Hunters_CNN.h5'
################################################################################
##########=           (Setup Code from Previous NBs)           =################
################################################################################
import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from scipy.signal import savgol_filter
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Load data
df_train = pd.read_csv('exoTrain.csv')
df_train['LABEL'] = df_train['LABEL'] - 1

# Load the CNN model
model = tf.keras.models.load_model('cnn')

def preprocess_data(df):
    """Apply preprocessing steps to the dataframe."""
    X = df.drop('LABEL', axis=1).values
    y = df['LABEL'].values
    # Fourier transform
    X = np.abs(np.fft.fft(X, axis=1))
    # Savitzky-Golay filter
    X = savgol_filter(X, 21, 4, deriv=0, axis=1)
    # Normalize
    X = normalize(X)
    # Robust scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    # SMOTE
    smote = SMOTE()
    X, y = smote.fit_resample(X, y)
    # Expand dimensions for CNN input
    X = np.expand_dims(X, axis=2)
    return X, y

X_train, y_train = preprocess_data(df_train)

def predict(index):
    """Make a prediction using the Keras model."""
    tensor = X_train[index].reshape(1, -1, 1)
    output = model.predict(tensor)
    return output.flatten()[0]

################################################################################
##########=                   (Confusion Matrix)               =################
################################################################################
st.title('Exoplanet Light Curve Visualization with CNN Predictions')

# Slider for selecting the index of the light curve
index = st.slider("Select Index for Light Curve", min_value=0, max_value=len(X_train)-1, value=12, step=1)

# Display CNN prediction results
prediction = predict(index)
st.write(f"Prediction (probability of being an exoplanet): {prediction:.4f}")

# Compute and plot confusion matrix
cnn_pred_labels = (model.predict(X_train).flatten() > 0.5).astype(int)
cm_cnn = confusion_matrix(y_train, cnn_pred_labels)

# Confusion Matrix for CNN
fig_cm_cnn = ff.create_annotated_heatmap(z=cm_cnn, x=['Pred 0', 'Pred 1'], y=['True 0', 'True 1'],
                                         colorscale='Viridis', showscale=True)
fig_cm_cnn.update_layout(title='Confusion Matrix for CNN')
st.plotly_chart(fig_cm_cnn, use_container_width=True)

Overwriting app.py


**Launch your website again:**

In [14]:
launch_website()

Click this link to try your web app:


ERROR:pyngrok.process.ngrok:t=2024-07-27T21:39:26+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/tunnels/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2024-07-27T21:39:26+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/tunnels/agents\r\n\r\nERR_NGROK_108\r\n.

### **Checkboxes in Streamlit**

To add another level of interactivity to Streamlit, we can add checkboxes that the user can interact with. To do this, we can set a variable like so to a Streamlit checkbox element:
```
show_full_light_curve = st.checkbox("Show Full Light Curve", value=True)
```

Using this value, we can set it so that the graph is only displayed when the user selects it with a checkbox
```
if show_full_light_curve:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=X_train[index], mode='lines', name='Light Curve'))
    fig.add_shape(type="rect",
                  x0=t_0, y0=min(X_train[index])-5,
                  x1=t_0+period, y1=max(X_train[index])+5,
                  line=dict(color="Red"),
                  fillcolor="LightPink", opacity=0.5)
    fig.update_layout(title="Box Covering One Period of Exoplanet Transit",
                      xaxis_title="Observation Point",
                      yaxis_title="Normalized Flux",
                      showlegend=True)
    st.plotly_chart(fig, use_container_width=True)
```

**Let's try this out with our full light curve, selected light curve, and confusion matrix!**

Your code should be structured similarly to this:
```
- Setup Code
- Show Prediction
- Checkboxes
- If statements for each graph
```

Tip: Try to work things step by step, starting with the light curves, and then the confusion matrix!

In [20]:
%%writefile app.py

model_path = 'Planet_Hunters_CNN.h5'
################################################################################
##########=           (Setup Code from Previous NBs)           =################
################################################################################

import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from scipy.signal import savgol_filter
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Load data
df_train = pd.read_csv('exoTrain.csv')
df_train['LABEL'] = df_train['LABEL'] - 1

# Load the CNN model
cnn_model = tf.keras.models.load_model('cnn')

def preprocess_data(df):
    """Apply preprocessing steps to the dataframe."""
    X = df.drop('LABEL', axis=1).values
    y = df['LABEL'].values
    # Fourier transform
    X = np.abs(np.fft.fft(X, axis=1))
    # Savitzky-Golay filter
    X = savgol_filter(X, 21, 4, deriv=0, axis=1)
    # Normalize
    X = normalize(X)
    # Robust scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    # SMOTE
    smote = SMOTE()
    X, y = smote.fit_resample(X, y)
    # Expand dimensions for CNN input
    X_cnn = np.expand_dims(X, axis=2)
    return X, X_cnn, y

X_train, X_train_cnn, y_train = preprocess_data(df_train)

def predict_cnn(index):
    """Make a prediction using the CNN model."""
    tensor = X_train_cnn[index].reshape(1, -1, 1)
    output = cnn_model.predict(tensor)
    return output.flatten()[0]

st.title('Exoplanet Light Curve Visualization with CNN Predictions')

################################################################################
##########=                   YOUR CODE BELOW                  =################
################################################################################

Overwriting app.py


In [52]:
#@title Instructor Solution

%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from scipy.signal import savgol_filter
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Load data
df_train = pd.read_csv('exoTrain.csv')
df_train['LABEL'] = df_train['LABEL'] - 1

# Load the CNN model
cnn_model = tf.keras.models.load_model('cnn')

def preprocess_data(df):
    """Apply preprocessing steps to the dataframe."""
    X = df.drop('LABEL', axis=1).values
    y = df['LABEL'].values
    # Fourier transform
    X = np.abs(np.fft.fft(X, axis=1))
    # Savitzky-Golay filter
    X = savgol_filter(X, 21, 4, deriv=0, axis=1)
    # Normalize
    X = normalize(X)
    # Robust scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    # SMOTE
    smote = SMOTE()
    X, y = smote.fit_resample(X, y)
    # Expand dimensions for CNN input
    X_cnn = np.expand_dims(X, axis=2)
    return X, X_cnn, y

X_train, X_train_cnn, y_train = preprocess_data(df_train)

def predict_cnn(index):
    """Make a prediction using the CNN model."""
    tensor = X_train_cnn[index].reshape(1, -1, 1)
    output = cnn_model.predict(tensor)
    return output.flatten()[0]

st.title('Exoplanet Light Curve Visualization with CNN Predictions')

# Slider for selecting the index of the light curve
index = st.slider("Select Index for Light Curve", min_value=0, max_value=len(X_train)-1, value=12, step=1)

# Display CNN prediction results
cnn_prediction = predict_cnn(index)
st.write(f"CNN Prediction (probability of being an exoplanet): {cnn_prediction:.4f}")

t_0 = st.slider("Start of Period (t_0)", min_value=0, max_value=3197, value=430, step=1)
period = st.slider("Length of Period", min_value=0, max_value=3197, value=1184, step=1)

# Checkbox widgets for graph selection
show_full_light_curve = st.checkbox("Show Full Light Curve", value=True)
show_selected_period = st.checkbox("Show Selected Period", value=True)
show_confusion_matrix = st.checkbox("Show Confusion Matrix", value=True)

# Plotly graph for full light curve with rectangle highlighting the period
if show_full_light_curve:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=X_train[index], mode='lines', name='Light Curve'))
    fig.add_shape(type="rect",
                  x0=t_0, y0=min(X_train[index])-5,
                  x1=t_0+period, y1=max(X_train[index])+5,
                  line=dict(color="Red"),
                  fillcolor="LightPink", opacity=0.5)
    fig.update_layout(title="Box Covering One Period of Exoplanet Transit",
                      xaxis_title="Observation Point",
                      yaxis_title="Normalized Flux",
                      showlegend=True)
    st.plotly_chart(fig, use_container_width=True)

# Plotly graph for just the selected period
if show_selected_period:
    fig_period = go.Figure()
    fig_period.add_trace(go.Scatter(y=X_train[index, t_0: t_0+period], mode='lines', name='Selected Period'))
    fig_period.update_layout(title="Plot of Just One Period",
                             xaxis_title="Observation Point",
                             yaxis_title="Normalized Flux",
                             showlegend=True)
    st.plotly_chart(fig_period, use_container_width=True)

# Compute and plot confusion matrices
cnn_pred_labels = (cnn_model.predict(X_train_cnn).flatten() > 0.5).astype(int)
cm_cnn = confusion_matrix(y_train, cnn_pred_labels)

# Confusion Matrix for CNN
if show_confusion_matrix:
    fig_cm_cnn = ff.create_annotated_heatmap(z=cm_cnn, x=['Pred 0', 'Pred 1'], y=['True 0', 'True 1'],
                                             colorscale='Viridis', showscale=True)
    fig_cm_cnn.update_layout(title='Confusion Matrix for CNN')
    st.plotly_chart(fig_cm_cnn, use_container_width=True)


Overwriting app.py


In [ ]:
launch_website()

Click this link to try your web app:
NgrokTunnel: "https://12dd-35-185-1-182.ngrok-free.app" -> "http://localhost:80"
2024-07-27 21:47:08.235834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 21:47:08.235926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 21:47:08.239692: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-27 21:47:09.967782: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
